In [ ]:
#|default_exp predictor

# Predictor.

> High level predictor.

In [9]:
#|export
from share import *
import config

import cv2
import einops
import gradio as gr
import numpy as np
import torch
import random

from pytorch_lightning import seed_everything
from annotator.util import resize_image, HWC3
from annotator.canny import CannyDetector
from cldm.model import create_model, load_state_dict
from cldm.ddim_hacked import DDIMSampler

In [24]:
#|export
class Predictor:
    def __init__(self, config_path, weight_path):
        self.apply_canny = CannyDetector()
        self.model = create_model(str(config_path)).cpu()
        self.model.load_state_dict(load_state_dict(str(weight_path), location='cuda'))
        self.model = self.model.cuda()
        self.ddim_sampler = DDIMSampler(self.model)

    def gen(input_image, prompts, **kwargs):
        for prompt in prompts:
            yield self.process(input_image, prompt, **kwargs)
                
    def process(self,
                input_image,
                prompt,
                a_prompt='',
                n_prompt='',
                num_samples=2,
                image_resolution=512,
                ddim_steps=20,
                guess_mode=False,
                strength=1.0,
                low_threshold=100,
                high_threshold=200,
                scale=9.0,
                seed=42,
                eta=0.0):
        with torch.no_grad():
            img = resize_image(HWC3(input_image), image_resolution)
            H, W, C = img.shape

            detected_map = self.apply_canny(img, low_threshold, high_threshold)
            detected_map = HWC3(detected_map)

            control = torch.from_numpy(detected_map.copy()).float().cuda() / 255.0
            control = torch.stack([control for _ in range(num_samples)], dim=0)
            control = einops.rearrange(control, 'b h w c -> b c h w').clone()

            if seed == -1:
                seed = random.randint(0, 65535)
            seed_everything(seed)

            if config.save_memory:
                self.model.low_vram_shift(is_diffusing=False)

            cond = {"c_concat": [control], "c_crossattn": [self.model.get_learned_conditioning([prompt + ', ' + a_prompt] * num_samples)]}
            un_cond = {"c_concat": None if guess_mode else [control], "c_crossattn": [self.model.get_learned_conditioning([n_prompt] * num_samples)]}
            shape = (4, H // 8, W // 8)

            if config.save_memory:
                self.model.low_vram_shift(is_diffusing=True)

            self.model.control_scales = [strength * (0.825 ** float(12 - i)) for i in range(13)] if guess_mode else ([strength] * 13)  # Magic number. IDK why. Perhaps because 0.825**12<0.01 but 0.826**12>0.01
            samples, intermediates = self.ddim_sampler.sample(ddim_steps, num_samples,
                                                         shape, cond, verbose=False, eta=eta,
                                                         unconditional_guidance_scale=scale,
                                                         unconditional_conditioning=un_cond)

            if config.save_memory:
                self.model.low_vram_shift(is_diffusing=False)

            x_samples = self.model.decode_first_stage(samples)
            x_samples = (einops.rearrange(x_samples, 'b c h w -> b h w c') * 127.5 + 127.5).cpu().numpy().clip(0, 255).astype(np.uint8)

            results = [x_samples[i] for i in range(num_samples)]
        return [255 - detected_map] + results

In [25]:
config_path = './models/cldm_v15.yaml'
weight_path = './models/control_sd15_canny.pth'

predictor = Predictor(config_path, weight_path) #config path is the yaml file

ControlLDM: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
Loaded model config from [./models/cldm_v15.yaml]
Loaded state_dict from [./models/control_sd15_canny.pth]


In [12]:
prompts_path = pv.download('https://gist.githubusercontent.com/lgvaz/8025ac53cc707e9d3222ab0153285fd5/raw/861a108626ee10dbdc6d18c07bef06cac5936824/prompts.txt', '.')
prompts = pv.open_txt(prompts_path).split('\n')

  0%|          | 0/5837 [00:00<?, ?B/s]

In [19]:
logo_files = pv.get_image_files(Path.home()/'logo')

In [32]:
import itertools
import polvo as pv
for prompt, logo_file in itertools.product(prompts, logo_files):
    break

In [26]:
for kwargs in pv.pbar(list(pv.kwargs_grid(prompt=prompts, logo_path=logo_files))):
    prompt = kwargs['prompt']
    logo_path = kwargs['logo_path']
    input_image = np.array(pv.open_image(logo_path))
    outs = predictor.process(input_image, prompt, seed=-1)
    
    save_dir = pv.mkdir(Path.home()/'control-output/', exist_ok=True)
    for i, out in enumerate(outs[1:]):
        save_file = save_dir/f'{logo_path.stem}-{prompt.replace(" ", "_")[:170]}-{i}.jpg'
        pv.save_image(out, save_file)

  0%|          | 0/540 [00:00<?, ?it/s]

Global seed set to 36769


Data shape for DDIM sampling is (2, 4, 64, 64), eta 0.0
Running DDIM Sampling with 20 timesteps



DDIM Sampler: 100%|██████████| 20/20 [00:12<00:00,  1.59it/s]
Global seed set to 29692


Data shape for DDIM sampling is (2, 4, 64, 64), eta 0.0
Running DDIM Sampling with 20 timesteps


...


DDIM Sampler: 100%|██████████| 20/20 [00:15<00:00,  1.33it/s]
Global seed set to 31808


Data shape for DDIM sampling is (2, 4, 64, 112), eta 0.0
Running DDIM Sampling with 20 timesteps



DDIM Sampler: 100%|██████████| 20/20 [00:30<00:00,  1.51s/it]
Global seed set to 33281


Data shape for DDIM sampling is (2, 4, 64, 96), eta 0.0
Running DDIM Sampling with 20 timesteps



DDIM Sampler:  85%|████████▌ | 17/20 [00:19<00:03,  1.16s/it]
IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [37]:
#|hide
from nbdev.export import nb_export
nb_export('predictor.ipynb', '.')